<a href="https://colab.research.google.com/github/SankirthGunnam/pet-breed-classifier/blob/main/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastai gradio

from fastai.vision.all import *
import os

# Download dataset
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

# Label function
def label_func(fname): return re.match(r'^([a-zA-Z_]+)', Path(fname).name)[0]

# DataLoaders
dls = ImageDataLoaders.from_name_func(
    path,
    files,
    label_func,
    item_tfms=Resize(224),
    bs=64
)

# Train model
learn = vision_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(1)

# Export model
export_path = "/tmp/model.pkl"
learn.export(export_path)

# Download to your local machine
from google.colab import files

if os.path.exists(export_path):
  files.download(export_path)
else:
  print(f"{export_path} not found.")

epoch,train_loss,valid_loss,accuracy,time
0,1.398942,0.293043,0.893099,00:39


epoch,train_loss,valid_loss,accuracy,time
0,0.446435,0.215689,0.927605,00:41


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import pathlib
import platform
from fastai.vision.all import *
import gradio as gr

# Handle path compatibility on Windows
if platform.system() == "Windows":
    pathlib.PosixPath = pathlib.WindowsPath

# Load the model
learn = load_learner("/tmp/model.pkl")


# Prediction function
def classify_image(img_path):
    try:
        img = PILImage.create(img_path)  # This is all you need
        pred, idx, probs = learn.predict(img)
        return dict(zip(learn.dls.vocab, map(float, probs)))
    except Exception as e:
        print(f"Error during prediction: {e}")
        return {}


# Gradio UI
gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="filepath", label="Upload a pet image"),
    outputs=gr.Label(num_top_classes=3, label="Top 3 Predictions"),
    title="🐶🐱 Pet Breed Classifier",
    description="Upload a dog or cat image to identify its breed using FastAI.",
).launch()

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4919a0299d5343c115.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
